In [1]:
import yfinance as yf
from sklearn.impute import KNNImputer
import pandas as pd

In [8]:

# Read ticker symbols from file
with open("tickers.txt", "r") as file:
    tickers = [line.strip() for line in file]

# Create an empty CSV file with headers to store data
output_file = "sp500_data.csv"
with open(output_file, "w") as f:
    f.write("Ticker,Date, Volume,Price_Change\n")  # Define headers

# Fetch historical data for all tickers and write iteratively
for ticker in tickers:
    try:
        print(f"Fetching data for {ticker}...")
        stock = yf.Ticker(ticker)
        index = stock.history(start="2010-01-01", end="2020-12-31")
        index['Price_Change'] = index['Close'].pct_change()  # Calculate percentage price change
        index.reset_index(inplace=True)  # Reset index to make 'Date' a column
        index['Ticker'] = ticker  # Add a column for the ticker symbol

        # Select relevant columns and write to the CSV file in append mode
        index[['Ticker', 'Date', 'Volume', 'Price_Change']].to_csv(
            output_file, mode='a', header=False, index=False
        )
    except Exception as e:
        print(f"Failed to fetch data for {ticker}: {e}")

print(f"Data collection complete! Saved to {output_file}.")

Fetching data for EQIX...
Fetching data for STLD...
Fetching data for GPC...
Fetching data for BKNG...
Fetching data for CSX...
Fetching data for AON...
Fetching data for BLK...
Fetching data for TJX...
Fetching data for IRM...
Fetching data for ELV...
Fetching data for ANET...
Fetching data for LVS...
Fetching data for PCG...
Fetching data for FAST...
Fetching data for GLW...
Fetching data for CNC...
Fetching data for TPR...
Fetching data for CL...
Fetching data for PWR...
Fetching data for FDX...
Fetching data for ROST...
Fetching data for WDC...
Fetching data for V...
Fetching data for AMD...
Fetching data for PM...
Fetching data for NVR...
Fetching data for J...
Fetching data for KDP...
Fetching data for TSLA...
Fetching data for PHM...
Fetching data for HD...
Fetching data for KEYS...
Fetching data for LW...
Fetching data for RCL...
Fetching data for DLTR...
Fetching data for ENPH...
Fetching data for COP...
Fetching data for ORCL...
Fetching data for PSX...
Fetching data for AEP.

In [14]:

# Load the CSV file
file_path = "sp500_data.csv"
data = pd.read_csv(file_path)

# Display the first few rows to check the structure
print(data.head())
print(data.shape)

missing_summary = data.isnull().sum()
print(missing_summary[missing_summary > 0])

# Convert 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])


# Sort data by Ticker and Date
data = data.sort_values(by=['Ticker', 'Date'])

# Group data by ticker and identify the start and end dates for each index
ticker_ranges = data.groupby('Ticker')['Date'].agg(['min', 'max']).reset_index()
print("Ticker ranges:", ticker_ranges)

# Function to filter rows for valid periods
def filter_valid_periods(group):
    print(f"Processing group for ticker: {group['Ticker'].iloc[0]}")  # Debug
    required_columns = ['Volume', 'Price_Change']
    existing_columns = [col for col in required_columns if col in group.columns]
    return group.dropna(subset=existing_columns)

# Process data group by group
cleaned_groups = []
for ticker, group in data.groupby('Ticker'):
    cleaned_group = filter_valid_periods(group)
    cleaned_groups.append(cleaned_group)

# Combine all cleaned groups into a single DataFrame
cleaned_data = pd.concat(cleaned_groups, axis=0).reset_index(drop=True)

# Count the number of records per ticker
ticker_counts = cleaned_data['Ticker'].value_counts()

# Remove tickers with fewer than a threshold (e.g., 1000 data points)
valid_tickers = ticker_counts[ticker_counts >= 1000].index
cleaned_data = cleaned_data[cleaned_data['Ticker'].isin(valid_tickers)]

# Save the cleaned data to a CSV file
cleaned_data.to_csv("cleaned_sp500_data.csv", index=False)
print("Cleaned data saved to cleaned_sp500_data.csv.")


  Ticker                       Date   Volume  Price_Change
0   EQIX  2010-01-04 00:00:00-05:00   576300           NaN
1   EQIX  2010-01-05 00:00:00-05:00   681900     -0.009310
2   EQIX  2010-01-06 00:00:00-05:00  1397500      0.009121
3   EQIX  2010-01-07 00:00:00-05:00   797200     -0.020451
4   EQIX  2010-01-08 00:00:00-05:00   432400     -0.004847
(1291674, 4)
Price_Change    480
dtype: int64
Ticker ranges:     Ticker                        min                        max
0        A  2010-01-04 00:00:00-05:00  2020-12-30 00:00:00-05:00
1      AAL  2010-01-04 00:00:00-05:00  2020-12-30 00:00:00-05:00
2     AAPL  2010-01-04 00:00:00-05:00  2020-12-30 00:00:00-05:00
3     ABBV  2013-01-02 00:00:00-05:00  2020-12-30 00:00:00-05:00
4      ABT  2010-01-04 00:00:00-05:00  2020-12-30 00:00:00-05:00
..     ...                        ...                        ...
475    YUM  2010-01-04 00:00:00-05:00  2020-12-30 00:00:00-05:00
476    ZBH  2010-01-04 00:00:00-05:00  2020-12-30 00:00:00-05:00


In [25]:
clean_data = pd.read_csv("cleaned_sp500_data.csv")
print(clean_data.shape)
missing_summary = clean_data.isnull().sum()
print(missing_summary[missing_summary > 0])

X = clean_data.drop(columns=['Price_Change'])
y = clean_data['Price_Change']
print(X.shape, y.shape)
print(X.head())
print(y.head())

(1291194, 4)
Series([], dtype: int64)
(1291194, 3) (1291194,)
  Ticker                       Date   Volume
0      A  2010-01-05 00:00:00-05:00  4186031
1      A  2010-01-06 00:00:00-05:00  3243779
2      A  2010-01-07 00:00:00-05:00  3095172
3      A  2010-01-08 00:00:00-05:00  3733918
4      A  2010-01-11 00:00:00-05:00  4781579
0   -0.010863
1   -0.003553
2   -0.001297
3   -0.000324
4    0.000649
Name: Price_Change, dtype: float64
